In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the dataset folder
# Set the path to the dataset folder
dataset_path = 'C:\\Users\\mujta\\Downloads\\BoneFractureDataset'



# Set the target image size for preprocessing and data augmentation
target_size = (256, 256)

# Create data generators for train and validation sets
train_data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    horizontal_flip=True
)

val_data_gen = ImageDataGenerator(rescale=1.0/255.0)

# Load the train and validation datasets from folders
train_generator = train_data_gen.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    target_size=target_size,
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_generator = val_data_gen.flow_from_directory(
    os.path.join(dataset_path, 'val'),
    target_size=target_size,
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [4]:
from tensorflow.keras import layers, models

# Build the deep learning model
def build_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Input shape for the model
input_shape = (*target_size, 3)

# Build the model
model = build_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)
model.save('C:\\Users\\mujta\\Downloads\\BoneFractureDataset\\model.h5')


Epoch 1/20
277/277 [==============================] - 1353s 5s/step - loss: 0.6147 - accuracy: 0.6526 - val_loss: 0.6290 - val_accuracy: 0.5850
Epoch 2/20
277/277 [==============================] - 1572s 6s/step - loss: 0.4070 - accuracy: 0.8146 - val_loss: 0.7900 - val_accuracy: 0.6733
Epoch 3/20
277/277 [==============================] - 1495s 5s/step - loss: 0.2187 - accuracy: 0.9131 - val_loss: 0.6845 - val_accuracy: 0.7933
Epoch 4/20
277/277 [==============================] - 1455s 5s/step - loss: 0.1156 - accuracy: 0.9601 - val_loss: 1.6572 - val_accuracy: 0.6567
Epoch 5/20
112/277 [===========>..................] - ETA: 14:55 - loss: 0.0877 - accuracy: 0.9685

KeyboardInterrupt: 

In [14]:
model.save('C:\\Users\\mujta\\Downloads\\BoneFractureDataset\\model.h5')

In [15]:
# Assuming you have a separate test set in a folder named 'test'
test_data_gen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_data_gen.flow_from_directory(
    os.path.join(dataset_path, 'C:\\Users\\mujta\\Downloads\\BoneFractureDataset\\val'),
    target_size=target_size,
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy: {test_accuracy}")


Found 600 images belonging to 2 classes.
19/19 [==============================] - 18s 945ms/step - loss: 2.3300 - accuracy: 0.6650
Test accuracy: 0.6650000214576721


In [16]:
# Function for displaying images
def display_image(image_path):
    image = cv2.imread('C:\\Users\\mujta\\Downloads\\BoneFractureDataset\\val\\fractured\\1-rotated1.jpg')
    cv2.imshow("Original Image", image)
    
    # Apply noise reduction (median filter)
    image = cv2.medianBlur(image, 5)
    cv2.imshow("Image with Noise Reduction", image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage: Call the function with the actual image path
display_image('C:\\Users\\mujta\\Downloads\\BoneFractureDataset\\val\\fractured\\1-rotated1.jpg')


In [28]:
import os
import cv2
import tkinter as tk
from tkinter import filedialog
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the dataset folder
dataset_path = 'C:\\Users\\mujta\\Downloads\\BoneFractureDataset'

# Set the target image size for preprocessing and data augmentation
target_size = (256, 256)

# Function for loading the trained model
def load_model():
    return tf.keras.models.load_model('C:\\Users\\mujta\\Downloads\\BoneFractureDataset\\model.h5')

# Function for preprocessing the uploaded image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Function for generating the report
def generate_report():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        image = preprocess_image(file_path)
        prediction = model.predict(image)[0][0]

        result_label.config(text=f"Result: {'Not Fractured' if prediction >= 0.5 else 'Fractured'}")

# Create the Tkinter window
window = tk.Tk()
window.title("Bone Fracture Detection")

# Load the model
model = load_model()

# Create a button to upload an image
upload_button = tk.Button(window, text="Upload Image", command=generate_report)
upload_button.pack(pady=20)

# Create a label to display the result
result_label = tk.Label(window, text="Result: ")
result_label.pack(pady=10)

# Start the Tkinter main loop
window.mainloop()


1/1 [==============================] - 0s 119ms/step
